In [1]:
import ROOT
from fitter import FitHandler

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x8723580


In [2]:
# TODO: make it possible to fit multiple runs simultaneously
run = {
    "lumi": 5000,
    # "lumi": 25000,
    "e_pol": 0.,
    "p_pol": 0.,
}

# https://arxiv.org/pdf/1506.07830
ilc_250_h20_lumi = 2000
ilc_250_h20 = [
    {
        "lumi": ilc_250_h20_lumi * 0.675,
        "e_pol": -0.8,
        "p_pol": 0.3,
    },
    {
        "lumi": ilc_250_h20_lumi * 0.225,
        "e_pol": 0.8,
        "p_pol": -0.3,
    },
    {
        "lumi": ilc_250_h20_lumi * 0.05,
        "e_pol": -0.8,
        "p_pol": -0.3,
    },
    {
        "lumi": ilc_250_h20_lumi * 0.05,
        "e_pol": 0.8,
        "p_pol": 0.3,
    },
]

# run = ilc_250_h20[0]

In [3]:
conf = {
    "parameters": {
        "g1z": 0.0,
        "ka": 0.0,
        "la": 0.0,
    },
    "obs_names": [
        "O_g1z_pos_1em05",
        "O_ka_pos_1em05",
        "O_la_pos_1em05",
    ],
    "signal_cat": "4f_sw_sl_signal",
    "signal_processes": [
        "4f_sw_sl_eLpL_signal",
        "4f_sw_sl_eLpR_signal",
        "4f_sw_sl_eRpL_signal",
        "4f_sw_sl_eRpR_signal",
    ],
    "backgrounds": [
        "4f_sl_bkg",
        "4f_not_sl",
        "2f",
        "3f",
        "5f",
        "6f",
        "higgs",
    ],
    "n_bins": 65,
}
input_path = "data/histograms/full/raw_histograms.root"

In [4]:
fit_handler = FitHandler(input_path, conf)

In [5]:
test_fun,_ = fit_handler.make_expected_fun(conf["obs_names"][0], run)

start constructor
finished constructor


In [6]:
pars = [1.] * 7
print(test_fun(pars))

-2558662.5922987997


In [7]:
model = fit_handler.build_model(run)
coupling_pars = fit_handler.coupling_pars

hello
start constructor
finished constructor
start constructor
finished constructor
start constructor
finished constructor


In [8]:
model.Print()

RooMultiVarGaussian::multi_gauss[ x=(O_g1z_pos_1em05,O_ka_pos_1em05,O_la_pos_1em05) mu=(expectation_O_g1z_pos_1em05,expectation_O_ka_pos_1em05,expectation_O_la_pos_1em05) ] = 1


In [9]:
# model.fitTo(ds)
# model.getVal()
model.Print("t")

0x226d1760 RooMultiVarGaussian::multi_gauss = 1 [Auto,Dirty] 
  0x21b369f0/V- RooRealVar::O_g1z_pos_1em05 = -328313
  0x4b0ed30/V- RooRealVar::O_ka_pos_1em05 = -816884
  0x21a0e6e0/V- RooRealVar::O_la_pos_1em05 = -404445
  0x22181280/V- RooFunctorBinding::expectation_O_g1z_pos_1em05 = -328313 [Auto,Clean] 
    0x20b8a4b0/V- RooRealVar::lumi = 1
    0x17126340/V- RooRealVar::e_pol = 1
    0x11fe0100/V- RooRealVar::p_pol = 1
    0x13724780/V- RooRealVar::g1z = 0
    0x21cb35b0/V- RooRealVar::ka = 0
    0x21c50a10/V- RooRealVar::la = 0
    0x21c4f9d0/V- RooRealVar::mu_signal = 1
  0x22595e00/V- RooFunctorBinding::expectation_O_ka_pos_1em05 = -816884 [Auto,Clean] 
    0x20b8a4b0/V- RooRealVar::lumi = 1
    0x17126340/V- RooRealVar::e_pol = 1
    0x11fe0100/V- RooRealVar::p_pol = 1
    0x13724780/V- RooRealVar::g1z = 0
    0x21cb35b0/V- RooRealVar::ka = 0
    0x21c50a10/V- RooRealVar::la = 0
    0x21c4f9d0/V- RooRealVar::mu_signal = 1
  0x22680620/V- RooFunctorBinding::expectation_O_la_pos_

In [10]:
def make_ttree_from_obs(obs):
    from array import array
    tree = ROOT.TTree("tree", "tree")
    branch_pointers = []
    for i, o in enumerate(obs_initial):
        name = obs_names[i]
        p = array("d", [0])
        tree.Branch(name, p, f"{name}/D")
        p[0] = o
        branch_pointers.append(p)
    tree.Fill()
    return tree

In [11]:
ds = ROOT.RooStats.AsymptoticCalculator.GenerateAsimovData(model, fit_handler.obs_pars)
ds.Print("v")

DataStore CountingAsimovData0 (CountingAsimovData0)
  Contains 1 entries
  Observables: 
    1)  O_g1z_pos_1em05 = -328313  L(-334268 - -322357)  "O_g1z_pos_1em05"
    2)   O_ka_pos_1em05 = -816884  L(-822088 - -811680)  "O_ka_pos_1em05"
    3)   O_la_pos_1em05 = -404445  L(-412478 - -396412)  "O_la_pos_1em05"


In [12]:
fit_res = model.fitTo(ds, Save=True)

[#1] INFO:Fitting -- RooAbsPdf::fitTo(multi_gauss_over_multi_gauss_Int[O_g1z_pos_1em05,O_ka_pos_1em05,O_la_pos_1em05]) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- using generic CPU library compiled with no vectorizations
[#1] INFO:Fitting -- Creation of NLL object took 5.44436 ms
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_multi_gauss_over_multi_gauss_Int[O_g1z_pos_1em05,O_ka_pos_1em05,O_la_pos_1em05]_CountingAsimovData0) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
Minuit2Minimizer: Minimize with max-calls 1500 convergence for edm < 1 strategy 1
Minuit2Minimizer : Valid minimum - status = 0
FVAL  = 25.5693367216680087
Edm   = 4.35788018137643373e-26
Nfcn  = 40
g1z	  = 0	 +/-  0.000880172	(limited)
ka	  = 0	 +/-  0.0011991	(limited)
la	  = 0	 +/-  0.000861372	(limited)
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setO

Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 220.186 μs
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =       25.56933672 Edm =   3.618396539e-26 NCalls =     13
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : 25.56933672
  Edm           : 3.618396539e-26
  Internal parameters:	[                0                0                0]	
  Internal gradient  :	[  -7.93969837e-11  9.704075785e-11  1.058626449e-10]	
  Internal covariance matrix:
[[  2.7922202e-06              0              0]
 [              0  1.0083714e-05              0]
 [              0              0  2.8711442e-06]]]
Info in <Minuit2>: VariableMetricBuilder Start iterating until Edm is < 0.001 with call limit = 1500
Info in <Minuit2>: VariableMetricBuilder    0 - FCN =       25.56933672 Edm =   3.618396539e-26 NCalls =     13
Warning in <Minuit2>: VariableMetricBuilder No improvemen

In [13]:
fit_res.covarianceMatrix().Print()


3x3 matrix is as follows

     |      0    |      1    |      2    |
--------------------------------------------
   0 |  7.747e-07  -2.769e-07   5.286e-07 
   1 | -2.769e-07   1.438e-06  -1.601e-08 
   2 |  5.286e-07  -1.601e-08    7.42e-07 



In [14]:
# abuse ConditionalObservables to inhibit integration
nll = model.createNLL(ds, EvalBackend="cpu")
# nll = model.createNLL(ds, EvalBackend="legacy")
# nll = model.createNLL(ds, EvalBackend="codegen")
# nll = model.createNLL(ds, EvalBackend="codegen_no_grad")
# not nll but chi2
# nll = chi2_fun

[#1] INFO:Fitting -- RooAbsPdf::fitTo(multi_gauss_over_multi_gauss_Int[O_g1z_pos_1em05,O_ka_pos_1em05,O_la_pos_1em05]) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- Creation of NLL object took 174.576 μs


In [15]:
nll_minimizer = ROOT.RooMinimizer(nll)

[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_multi_gauss_over_multi_gauss_Int[O_g1z_pos_1em05,O_ka_pos_1em05,O_la_pos_1em05]_CountingAsimovData0) Summation contains a RooNLLVar, using its error level


In [16]:
%time
nll_minimizer.migrad()

CPU times: user 5 μs, sys: 1 μs, total: 6 μs
Wall time: 10.7 μs


0

Minuit2Minimizer: Minimize with max-calls 1500 convergence for edm < 1 strategy 1
Minuit2Minimizer : Valid minimum - status = 0
FVAL  = 25.5693367216680087
Edm   = 4.01127747495378121e-25
Nfcn  = 40
g1z	  = 0	 +/-  0.000880172	(limited)
ka	  = 0	 +/-  0.0011991	(limited)
la	  = 0	 +/-  0.000861372	(limited)


Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 269.913 μs
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =       25.56933672 Edm =   2.538401714e-25 NCalls =     13
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : 25.56933672
  Edm           : 2.538401714e-25
  Internal parameters:	[                0                0                0]	
  Internal gradient  :	[                0  -2.24380401e-10 -4.205016199e-10]	
  Internal covariance matrix:
[[  2.7922202e-06              0              0]
 [              0  1.0083714e-05              0]
 [              0              0  2.8711442e-06]]]
Info in <Minuit2>: VariableMetricBuilder Start iterating until Edm is < 0.001 with call limit = 1500
Info in <Minuit2>: VariableMetricBuilder    0 - FCN =       25.56933672 Edm =   2.538401714e-25 NCalls =     13
Warning in <Minuit2>: VariableMetricBuilder No improvemen

In [17]:
%time
# nll_minimizer.hesse()
# nll_minimizer.minos(coupling_pars)

CPU times: user 3 μs, sys: 1 μs, total: 4 μs
Wall time: 6.91 μs


In [18]:
# nll.Print("t")

In [19]:
pll0 = nll.createProfile({coupling_pars[0]})
pll1 = nll.createProfile({coupling_pars[1]})
pll2 = nll.createProfile({coupling_pars[2]})

In [20]:
frame0 = coupling_pars[0].frame(Range=(-0.005, 0.005))
frame1 = coupling_pars[1].frame(Range=(-0.005, 0.005))
frame2 = coupling_pars[2].frame(Range=(-0.005, 0.005))
nll.plotOn(frame0, ShiftToZero=True)
nll.plotOn(frame1, ShiftToZero=True)
nll.plotOn(frame2, ShiftToZero=True)

In [21]:
pll0.plotOn(frame0, LineColor="r")
pll1.plotOn(frame1, LineColor="r")
pll2.plotOn(frame2, LineColor="r")

[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[g1z]) Creating instance of MINUIT
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_multi_gauss_over_multi_gauss_Int[O_g1z_pos_1em05,O_ka_pos_1em05,O_la_pos_1em05]_CountingAsimovData0) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[g1z]) determining minimum likelihood for current configurations w.r.t all observable
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[g1z]) minimum found at (g1z=0)
..........................................................................................................................................................................................................
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[ka]) Creating instance of MINUIT
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_multi_gauss_over_multi_gauss_Int[O_g1z_

In [22]:
c0 = ROOT.TCanvas()
frame0.SetMinimum(0)
frame0.SetMaximum(4)
frame0.Draw()
c0.Draw()
# c0.SaveAs("plots/fit/ll_pll.pdf(")

c1 = ROOT.TCanvas()
frame1.SetMinimum(0)
frame1.SetMaximum(4)
frame1.Draw()
c1.Draw()
# c1.SaveAs("plots/fit/ll_pll.pdf")

c2 = ROOT.TCanvas()
frame2.SetMinimum(0)
frame2.SetMaximum(4)
frame2.Draw()
c2.Draw()
# c2.SaveAs("plots/fit/ll_pll.pdf)")